In [37]:
from pydrake.examples.pendulum import PendulumParams
import control
import numpy as np
p = PendulumParams()

# Add the Linearization
A = np.array([[0, 1], [p.mass()*p.gravity()*p.length(), -p.damping()]])
B = np.array([[0], [1]])
E = np.array([[1, 0], [0, p.mass()*p.length()*p.length()]])

# Add LQR Costs
Q = np.diag([10, 1])
R = [1]

# matrix needed for algorithm
S = np.array([[0], [0]])

X, L, G = control.care(A, B, Q, R, S, E)

A_closed = A - B@G
print(A_closed)
Q_ly = np.eye(2)
P = control.lyap(A_closed.T, Q_ly, E = E.T)
candidate_matrix = E.T@P@E

[[ 0.          1.        ]
 [-5.83601105 -2.52596626]]


In [38]:
candidate_matrix

array([[1.42110175, 0.02141874],
       [0.02141874, 0.05160587]])

In [27]:
candidate_matrix_new = np.array([[0.0515, 0, 0.021, 0], [0, 0, 0, -1.42], [0.021, 0, 0, 0], [0, -1.42, 0, 2.8422]])

In [28]:
G_casted = np.insert(G, 1, 0)

In [67]:
# finding the bound

from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver
from pydrake.all import Polynomial, ToLatex, Variables
from IPython.display import Markdown, display
import math

negative_infinity = float('-inf')
coeff = 1e-2

def find_implicit_lambdas(x, xd, V, Vdot, g, x0):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
    rho = prog.NewContinuousVariables(1, 'rho')[0]
        
    y = np.hstack([x, xd])
    
    l_deg = math.ceil(Vdot.TotalDegree()/2) * 2
    
    lambda_ = prog.NewSosPolynomial(Variables(y), l_deg)[0]
    
    lambda_g = []
    
    for i in range(g.size): 
        gi_deg = max(l_deg + Vdot.TotalDegree() - g[i].TotalDegree(), 0)
        lambda_g.append(prog.NewFreePolynomial(Variables(y), gi_deg))
        
    lambda_g = np.array(lambda_g)    
    
    s_deg = max(l_deg + Vdot.TotalDegree() - 2, 0)
    lambda_s = prog.NewFreePolynomial(Variables(y), s_deg)
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)    
    # level set as optimization variable
    prog.AddSosConstraint(Polynomial((x-x0).dot(x-x0))*(V - rho) -lambda_*Vdot + lambda_g.dot(g) + lambda_s*trig)
    prog.AddLinearCost(-rho)
        
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("Lambda solution status: " + str(k))
    
    assert result.is_success()
    
    lambda_g_results = []
    
    for i in range(g.size):
        lambda_g_results.append(result.GetSolution(lambda_g[i]).RemoveTermsWithSmallCoefficients(coeff))
    
    return result.GetSolution(lambda_).RemoveTermsWithSmallCoefficients(coeff), np.array(lambda_g_results), \
result.GetSolution(lambda_s).RemoveTermsWithSmallCoefficients(coeff), result.GetSolution(rho)


def problem_solver_implicit(S, V_degree, G):
    prog = MathematicalProgram()
    s = prog.NewIndeterminates(1, "s")
    c = prog.NewIndeterminates(1, "c")
    theta_dot = prog.NewIndeterminates(1, "theta_dot")
    
    xd = prog.NewIndeterminates(3, 'xd')
    
    state_V = np.hstack([theta_dot, c, s, [1]])
    
    x = np.hstack([s, c, theta_dot])
    x0 = np.array([0, 1, 0])
    u = -G@(x-x0)
        
    g0 = xd[0] - c[0]*theta_dot[0]
    g1 = xd[1] + s[0]*theta_dot[0]
    g2 = p.mass()*(p.length()**2)*xd[2] + p.damping()*theta_dot[0] - p.mass()*p.gravity()*p.length()*s[0] - u
    
    g = np.array([g0, g1, g2])
    
    V = state_V.T@S@state_V
    V = Polynomial(V)
    V = 1.4211*(2 - 2*x[1]) + 0.0515*x[2]**2 + 0.042*x[0]*x[2]
    Vdot = V.Jacobian(x).dot(xd)
    
    #Vdot = Polynomial(Vdot.ToExpression())

    
    V = Polynomial(V)
    Vdot = Polynomial(Vdot)
    
    print(Vdot)
    
    g_poly = []
    for i in range(g.size):
        g_poly.append(Polynomial(g[i]))
        
    g_poly = np.array(g_poly)
    
    display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
    for i in range(1):
        lambda_, lambda_g, lambda_s, rho = find_implicit_lambdas(x, xd, V, Vdot, g_poly, x0)
        
#         display(Markdown("$ \lambda(x)="+ToLatex(lambda_s.ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[0].ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[1].ToExpression())+"$"))
        
        #V, Q = find_implicit_V(x, xd, lambda_, lambda_g, lambda_s, g_poly, V_degree)
        print(rho)
        
        Vdot = V.Jacobian(x).dot(xd)
     
        display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
        
    return V, Q

V_degree = 2
V, Q = problem_solver_implicit(candidate_matrix_new, V_degree, G_casted)

-2.8422000000000001*xd(1) + 0.10299999999999999*theta_dot(0) * xd(2) + 0.042000000000000003*theta_dot(0) * xd(0) + 0.042000000000000003*s(0) * xd(2)


$ V(x)=(2.842200000 - 2.842200000c_{0} + 0.042000000s_{0} theta_dot_{0} + 0.051500000theta_dot_{0}^{2})$

Lambda solution status: 1
5.684399911327827


$ V(x)=(2.842200000 - 2.842200000c_{0} + 0.042000000s_{0} theta_dot_{0} + 0.051500000theta_dot_{0}^{2})$

In [77]:
# finding the bound

from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver
from pydrake.all import Polynomial, ToLatex, Variables
from IPython.display import Markdown, display
import math

negative_infinity = float('-inf')
coeff = 1e-2

def find_implicit_lambdas(x, xd, V, Vdot, g, x0, rho):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
        
    y = np.hstack([x, xd])
    
    l_deg = math.ceil(Vdot.TotalDegree()/2) * 2
    
    lambda_ = prog.NewSosPolynomial(Variables(y), l_deg)[0]
    
    lambda_g = []
    
    for i in range(g.size): 
        gi_deg = max(l_deg + Vdot.TotalDegree() - g[i].TotalDegree(), 0)
        lambda_g.append(prog.NewFreePolynomial(Variables(y), gi_deg))
        
    lambda_g = np.array(lambda_g)    
    
    s_deg = max(l_deg + Vdot.TotalDegree() - 2, 0)
    lambda_s = prog.NewFreePolynomial(Variables(y), s_deg)
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)    
    # level set as optimization variable
    #prog.AddSosConstraint(Polynomial((x-x0).dot(x-x0))*(V - rho) -lambda_*Vdot + lambda_g.dot(g) + lambda_s*trig)
    prog.AddLinearCost(-rho)
    
    eps = 1e-3
    prog.AddSosConstraint(- Vdot - lambda_ * (rho - V) - Polynomial(eps*(x-x0).dot(x-x0)) \
                          + lambda_g.dot(g) + lambda_s*trig)
        
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("Lambda solution status: " + str(k))
    
    assert result.is_success()
    
    lambda_g_results = []
    
    for i in range(g.size):
        lambda_g_results.append(result.GetSolution(lambda_g[i]).RemoveTermsWithSmallCoefficients(coeff))
    
    return result.GetSolution(lambda_).RemoveTermsWithSmallCoefficients(coeff), np.array(lambda_g_results), \
result.GetSolution(lambda_s).RemoveTermsWithSmallCoefficients(coeff)


def problem_solver_implicit(S, V_degree, G):
    prog = MathematicalProgram()
    s = prog.NewIndeterminates(1, "s")
    c = prog.NewIndeterminates(1, "c")
    theta_dot = prog.NewIndeterminates(1, "theta_dot")
    
    xd = prog.NewIndeterminates(3, 'xd')
    
    state_V = np.hstack([theta_dot, c, s, [1]])
    
    x = np.hstack([s, c, theta_dot])
    x0 = np.array([0, 1, 0])
    u = -G@(x-x0)
        
    g0 = xd[0] - c[0]*theta_dot[0]
    g1 = xd[1] + s[0]*theta_dot[0]
    g2 = p.mass()*(p.length()**2)*xd[2] + p.damping()*theta_dot[0] - p.mass()*p.gravity()*p.length()*s[0] - u
    
    g = np.array([g0, g1, g2])
    
    V = state_V.T@S@state_V
    V = Polynomial(V)
    V = 1.4211*(2 - 2*x[1]) + 0.0515*x[2]**2 + 0.042*x[0]*x[2]
    Vdot = V.Jacobian(x).dot(xd)
    
    #Vdot = Polynomial(Vdot.ToExpression())

    
    V = Polynomial(V)
    Vdot = Polynomial(Vdot)
        
    g_poly = []
    for i in range(g.size):
        g_poly.append(Polynomial(g[i]))
        
    g_poly = np.array(g_poly)
    
    rho = 0
    
    display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
    for i in range(100):
        lambda_, lambda_g, lambda_s = find_implicit_lambdas(x, xd, V, Vdot, g_poly, x0, rho)
        
#         display(Markdown("$ \lambda(x)="+ToLatex(lambda_s.ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[0].ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[1].ToExpression())+"$"))
        
        #V, Q = find_implicit_V(x, xd, lambda_, lambda_g, lambda_s, g_poly, V_degree)
        print(rho)
        
        rho += 0.1
    return V

V_degree = 2
V = problem_solver_implicit(candidate_matrix_new, V_degree, G_casted)

$ V(x)=(2.842200000 - 2.842200000c_{0} + 0.042000000s_{0} theta_dot_{0} + 0.051500000theta_dot_{0}^{2})$

Lambda solution status: 1
0
Lambda solution status: 1
0.1
Lambda solution status: 1
0.2
Lambda solution status: 1
0.30000000000000004
Lambda solution status: 1
0.4
Lambda solution status: 1
0.5
Lambda solution status: 1
0.6
Lambda solution status: 1
0.7
Lambda solution status: 1
0.7999999999999999
Lambda solution status: 1
0.8999999999999999
Lambda solution status: 1
0.9999999999999999
Lambda solution status: 1
1.0999999999999999
Lambda solution status: 1
1.2
Lambda solution status: 1
1.3
Lambda solution status: 1
1.4000000000000001
Lambda solution status: 1
1.5000000000000002
Lambda solution status: 1
1.6000000000000003
Lambda solution status: 1
1.7000000000000004
Lambda solution status: 1
1.8000000000000005
Lambda solution status: 1
1.9000000000000006
Lambda solution status: 1
2.0000000000000004
Lambda solution status: 1
2.1000000000000005
Lambda solution status: 1
2.2000000000000006
Lambda solution status: 1
2.3000000000000007
Lambda solution status: 1
2.400000000000001
Lambda solut

In [54]:
# finding the control input itself using alternations

from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

negative_infinity = float('-inf')
coeff = 1e-5

def find_implicit_lambdas(x, xd, V, Vdot, g):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
            
    y = np.hstack([x, xd])
    
    l_deg = math.ceil(V.TotalDegree()/2) * 2
    
    lambda_ = prog.NewSosPolynomial(Variables(y), l_deg)[0]
        
    lambda_g = []
    
    for i in range(g.size): 
        gi_deg = max(l_deg + V.TotalDegree() - g[i].TotalDegree(), 0)
        lambda_g.append(prog.NewFreePolynomial(Variables(y), gi_deg))
        
    lambda_g = np.array(lambda_g)    
    
    s_deg = max(l_deg + V.TotalDegree() - 2, 0)
    lambda_s = prog.NewFreePolynomial(Variables(y), s_deg)
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)
  
    prog.AddSosConstraint(-Vdot + lambda_*(V - 0.001) + lambda_g.dot(g) - lambda_s*trig)
        
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("Lambda solution status: " + str(k))
    
    assert result.is_success()
    
    lambda_g_results = []
    
    for i in range(g.size):
        lambda_g_results.append(result.GetSolution(lambda_g[i]).RemoveTermsWithSmallCoefficients(coeff))
    
    return result.GetSolution(lambda_).RemoveTermsWithSmallCoefficients(coeff), np.array(lambda_g_results), \
result.GetSolution(lambda_s).RemoveTermsWithSmallCoefficients(coeff)

def find_implicit_V(x, xd, lambda_, lambda_g, lambda_s, g, V_degree):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
            
    V, Q = prog.NewSosPolynomial(Variables(x), V_degree)
    Vdot = V.Jacobian(x).dot(xd)
    
    # the u business
    u = prog.NewFreePolynomial(Variables(x), 1)
    
    g2 = Polynomial(p.mass()*(p.length()**2)*xd[2] + p.damping()*x[2] \
                    - p.mass()*p.gravity()*p.length()*x[0]) - u
    g[2] = g2
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)
    prog.AddSosConstraint(-Vdot + lambda_*(V - 0.001) + lambda_g.dot(g) - lambda_s*trig)
    
    mgl = p.mass()*p.length()*p.gravity()
    
    prog.AddLinearCost(np.trace(Q))    
    prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 1, x[1]:1, x[2]:0}) == 0)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("V solution status: " + str(k))
    
    assert result.is_success()
        
    return result.GetSolution(V), result.GetSolution(Q), result.GetSolution(u)


def problem_solver_implicit(S, V_degree, G):
    prog = MathematicalProgram()
    s = prog.NewIndeterminates(1, "s")
    c = prog.NewIndeterminates(1, "c")
    theta_dot = prog.NewIndeterminates(1, "theta_dot")
    
    xd = prog.NewIndeterminates(3, 'xd')
    
    state_V = np.hstack([theta_dot, c, s, [1]])
    
    x = np.hstack([s, c, theta_dot])
    
    u = -G@x
        
    g0 = xd[0] - x[1]*x[2]
    g1 = xd[1] + x[0]*x[2]
    g2 = p.mass()*(p.length()**2)*xd[2] + p.damping()*x[2] - p.mass()*p.gravity()*p.length()*x[0] - u
    
    g = np.array([g0, g1, g2])
    
    V = state_V.T@S@state_V
    Vdot = V.Jacobian(x).dot(xd)
    
    V = Polynomial(V)
    Vdot = Polynomial(Vdot)
    
    g_poly = []
    for i in range(g.size):
        g_poly.append(Polynomial(g[i]))
        
    g_poly = np.array(g_poly)
    
    display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
    for i in range(1):
        lambda_, lambda_g, lambda_s = find_implicit_lambdas(x, xd, V, Vdot, g_poly)
        
        display(Markdown("$ \lambda(x)="+ToLatex(lambda_s.ToExpression())+"$"))
        display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[0].ToExpression())+"$"))
        display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[1].ToExpression())+"$"))
        
        V, Q, u = find_implicit_V(x, xd, lambda_, lambda_g, lambda_s, g_poly, V_degree)
        
        g2 = Polynomial(p.mass()*(p.length()**2)*xd[2] + p.damping()*x[2] \
                    - p.mass()*p.gravity()*p.length()*x[0]) - u
        g_poly[2] = g2
        
        Vdot = V.Jacobian(x).dot(xd)
     
        display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
        
        display(Markdown("$ u(x)="+ToLatex(u.ToExpression(), 9)+"$"))
        
    return V, Q, u, x

V_degree = 2
V, Q, u, x = problem_solver_implicit(candidate_matrix_new, V_degree, G_casted)

$ V(x)=(2.842200000 - 2.840000000c_{0} + 0.042000000s_{0} theta_dot_{0} + 0.051500000theta_dot_{0}^{2})$

0.25*xd(2) + 2.5259662559187128*theta_dot(0) + 5.8360110520799973*s(0)
Lambda solution status: 1


$ \lambda(x)=(-1.897 - 1.816c_{0} - 0.889xd_{1} + 0.610s_{0} theta_dot_{0} + 0.270s_{0} xd_{0} + 0.054s_{0} xd_{2} - 0.016c_{0} xd_{1} - 0.674theta_dot_{0} xd_{0} + 0.022theta_dot_{0} xd_{2} + 0.001xd_{0} xd_{2} - 0.366s_{0}^{2} - 0.923c_{0}^{2} - 1.435theta_dot_{0}^{2} - 1.658xd_{0}^{2} - 1.710xd_{1}^{2} - 1.460xd_{2}^{2})$

$ \lambda_g(x)=(0.066s_{0} - 0.283theta_dot_{0} - 0.034xd_{0} + 0.043xd_{2} - 0.290s_{0} c_{0} + 0.230s_{0} xd_{1} + 0.602c_{0} theta_dot_{0} + 1.537c_{0} xd_{0} + 0.035c_{0} xd_{2} - 0.428theta_dot_{0} xd_{1} + 0.014xd_{0} xd_{1} + 0.003xd_{1} xd_{2})$

$ \lambda_g(x)=(-1.147 - 0.960c_{0} - 0.247xd_{1} + 0.294s_{0} theta_dot_{0} - 0.185s_{0} xd_{0} + 0.018s_{0} xd_{2} + 2.008c_{0} xd_{1} + 0.386theta_dot_{0} xd_{0} - 0.044theta_dot_{0} xd_{2} - 0.009xd_{0} xd_{2} - 0.138s_{0}^{2} - 0.730c_{0}^{2} - 0.711theta_dot_{0}^{2} - 0.014xd_{0}^{2} + 0.000xd_{2}^{2})$

V solution status: 1


$ V(x)=(4.115837186 - 3.043763979s_{0} - 5.187800935c_{0} + 0.646089818theta_dot_{0} + 1.912069831s_{0} c_{0} - 0.208942260s_{0} theta_dot_{0} - 0.437138380c_{0} theta_dot_{0} + 0.565826895s_{0}^{2} + 1.637831004c_{0}^{2} + 0.097992954theta_dot_{0}^{2})$

$ u(x)=(4.266366346 - 7.897190086s_{0} - 2.631293755c_{0} - 1.633248581theta_dot_{0})$